# AFF - Generating Randomized Treated Landscape Scenarios
## Creates Randomized Treatment Scenario Landscapes - exports as ee.Images into an ee.ImageCollection

In [1]:
import ee
import geemap
import os
import random
import numpy as np
import math
from datetime import datetime
ee.Initialize()

## USER INPUT - Review/edit paths to your AOIs

In [2]:
parcels = ee.FeatureCollection("projects/aff-treatments/assets/AllParcels_forSIG")
NIparcels = ee.FeatureCollection("projects/aff-treatments/assets/AllNIPParcels_forSIG")
fireshed = ee.FeatureCollection("projects/aff-treatments/assets/SonoraFireshed")
parcels_dissolve = ee.FeatureCollection("projects/aff-treatments/assets/NIPparcels_dissolve")

aoi = fireshed

## USER INPUT - Review/edit the parameters for the treatment randomization procedure

In [3]:
TOTAL_AREA = 245278 #acres of AOI

# range of pct of total areas you want to simulate in scenarios
SCENARIO_PCT_RANGE = [0.05,0.6] # make as static list ascending not randomized b/w range

# total scenarios to run
SCENARIOS = 60 

# total number of distinct treatment sizes to use in the scenarios
TRT_SIZE_CLASSES = 3 # currently this is not dynamic, either make dynamic or take out as a User input

# smallest and largest acreage sizes you want the program to choose for treatment units
SIZE_CLASS_RANGE = [25,500]  # option for user defined list of size classes also
                                # select from a user-defined distro certain size classes in the SIZE_CLASS_RANGE

# LANDFIRE DIST CODE to assign to all treated pixels
DIST_CODE = 222

## Required Functions, Run and Continue

In [4]:
random.seed(8)

# base python math functions
def randn(low,i_sum,k):
    a = np.random.rand(k) #random floats
    weights = (a/a.sum()*(i_sum-low*k)) # adjust floats to sum to i_sum, with given low val threshold
    adjusted = weights+low # sum(adjusted) should == i_sum
    return [round(i,4) for i in list(adjusted)]

def treatment_math(TOTAL_AREA,SCENARIO_PCT_RANGE,SCENARIOS,TRT_SIZE_CLASSES,SIZE_CLASS_RANGE):
    """Returns the amount of treatment units needed per size class and the radii of the kernel needed for each size class to make the randomized treatment landscape in EE
    # args:
    # TOTAL_AREA (int): total area of aoi to consider treatments in acres
    # SCENARIO_PCT_RANGE (list): min and max range of possible pct treated area floats to be randomly chosen per scenario
    # SCENARIOS (int): Total unique scenarios to run
    # TRT_SIZE_CLASSES (int): number of unique treatment sizes to consider for all scenarios
    # SIZE_CLASS_RANGE (list): min and max range of possible treatment sizes to choose from 
    """
    # make list of length SCENARIOS with random floating point numbers in the range defined by SCNEARIO_PCT_RANGE (list of different total pct treated scenarios)
    scn=[]
    trt_areas=[]
    trt_props=[]
    sizes=[]
    intv = 25 # defining list of possible treatment sizes from user-provided range
    possible_sizes = list(range(SIZE_CLASS_RANGE[0],SIZE_CLASS_RANGE[1]+intv,intv))
    units=[]
    radii=[]
    
    for i in list(range(SCENARIOS)):
        rnd = round(random.uniform(SCENARIO_PCT_RANGE[0], SCENARIO_PCT_RANGE[1]), 4)
        scn.append(rnd)

        # compute total area to be treated for each scenario
        trt_areas_i = TOTAL_AREA*rnd#scn[i]
        trt_areas.append(trt_areas_i)

        # for each scenario random float, make list of length TRT_SIZE_CLASSES containing random floats that sum to scn[i] (ratios of pct are treated between the different sized treatment units) 
        i_prop = randn(0.001,trt_areas_i,TRT_SIZE_CLASSES) 
        trt_props.insert(i,i_prop)

        # randomly select list of treatment unit sizes of length TRT_SIZE_CLASSES from the range of possible sizes
        sizes_i = random.sample(possible_sizes,TRT_SIZE_CLASSES)
        sizes.insert(i,sizes_i)


        # get appoximate count of total treatment units per size class
        units_i = [int(round(j)) for j in list(np.divide(i_prop,sizes_i))]
        units.insert(i,units_i)

        # get approximate radius in meters needed for each size class to make correct-sized treatment units
        acres_to_sqm = [int(round(size_j*4027)) for size_j in sizes_i] # convert to sq meters for each size class in acreage
        radii_i = [int(round(math.sqrt(acreage/math.pi))) for acreage in acres_to_sqm] # solve for r: A = pi(r^2)
        radii.insert(i,radii_i)
    
    return scn,trt_areas,trt_props,sizes,units,radii

# EE functions
def distanceFilter(pts,distance):
    withinDistance = distance; ##go with pretty far apart at first

    ## From the User Guide: https:#developers.google.com/earth-engine/joins_spatial
    ## add extra filter to eliminate self-matches
    distFilter = ee.Filter.And(ee.Filter.withinDistance(**{
      'distance': withinDistance,
      'leftField': '.geo',
      'rightField': '.geo', 
      'maxError': 1
    }), ee.Filter.notEquals(**{
      'leftField': 'system:index',
      'rightField': 'system:index',

    }));
    
    distSaveAll = ee.Join.saveAll(**{
                  'matchesKey': 'points',
                  'measureKey': 'distance'
    });
    # Apply the join.
    spatialJoined = distSaveAll.apply(pts, pts, distFilter);

    # Check the number of matches.
    # We're only interested if nmatches > 0.
    spatialJoined = spatialJoined.map(lambda f: f.set('nmatches', ee.List(f.get('points')).size()) );
    spatialJoined = spatialJoined.filterMetadata('nmatches', 'greater_than', 0);

    # The real matches are only half the total, because if p1.withinDistance(p2) then p2.withinDistance(p1)
    # Use some iterative logic to clean up
    def unpack(l): 
        return ee.List(l).map(lambda f: ee.Feature(f).id())

    def iterator_f(f,list):
        key = ee.Feature(f).id()
        list = ee.Algorithms.If(ee.List(list).contains(key), list, ee.List(list).cat(unpack(ee.List(f.get('points')))))
        return list
    
    ids = spatialJoined.iterate(iterator_f,ee.List([]))
    ##print("Removal candidates' IDs", ids);

    # Clean up 
    cleaned_pts = pts.filter(ee.Filter.inList('system:index', ids).Not());
    return cleaned_pts

def ee_treatments(units,radii):
    seed = 5155115
    ptsBig = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,units[0]+(units[0]*2),seed,True,4,True) 
    ptsBig = distanceFilter(ptsBig,radii[0]*2.1).limit(units[0])
    #print('Big trt points',ptsBig.size().getInfo())
    areasBig = ptsBig.reduceToImage(['constant'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[0],'meters')).eq(1)

    areasBigmask = areasBig.distance(ee.Kernel.euclidean(radii[0]*2,'meters')).gte(0).Not().unmask(1)

#     Map.addLayer(ptsBig,{},'big pts')
#     Map.addLayer(areasBig, pal, 'big treatments')
#     Map.addLayer(areasBigmask,pal,'areasBigmask')
    
    ptsMedium = areasBigmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[1]+(units[1]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsMedium = distanceFilter(ptsMedium,radii[1]*2.1).limit(units[1])
    #print('Medium trt points',ptsMedium.size().getInfo())
    areasMedium = ptsMedium.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[1],'meters')).eq(1)

    areasMediummask = areasMedium.distance(ee.Kernel.euclidean(radii[1]*2,'meters')).gte(0).Not().unmask(1)
    blendmask = areasBigmask.add(areasMediummask).lte(1).Not().unmask(1)


    
    ptsSmall = blendmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[2]+(units[2]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsSmall = distanceFilter(ptsSmall,radii[2]*2.1).limit(units[2])
    #print('Small trt points',ptsSmall.size().getInfo())
    areasSmall = ptsSmall.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[2],'meters')).eq(1)
    
#     Map.addLayer(ptsSmall, {}, 'small points')
#     Map.addLayer(areasSmall, pal, 'small treatments')
    
    blendTreatments = areasBig.add(areasMedium).add(areasSmall).gte(1).multiply(DIST_CODE).selfMask().rename('DIST')
    
#     Map.addLayer(blendTreatments, pal,'final treatment landscape')

    return ee.Image(blendTreatments)

def export_img(img,imgcoll_p,aoi):
    """Export image to imageCollection"""
    desc = f"scenario{ee.String(ee.Image(img).getNumber('scenario').format()).getInfo()}"
    
    task = ee.batch.Export.image.toAsset(
        image=ee.Image(img),
        description=desc,
        assetId=f'{imgcoll_p}/{desc}', 
        region=aoi.geometry().bounds(), 
        scale=30, 
        crs='EPSG:5070', 
        maxPixels=1e13)

    task.start()
    print(f"Export Started for {imgcoll_p}/{desc}")
    

## Construct random treatment landscapes as rasters - format: an ee.Image exported to an ee.ImageCollection

In [5]:
# Run treatment math to construct your lists (length of SCENARIO) of the needed parameters
scn,trt_areas,trt_props,sizes,units,radii = treatment_math(TOTAL_AREA,SCENARIO_PCT_RANGE,SCENARIOS,TRT_SIZE_CLASSES,SIZE_CLASS_RANGE)

print('Example Scenario')
print('Total Area of AOI (ac): ', TOTAL_AREA)
print('scenario pct area: ',scn[0])
print('total area (ac) to treat: ', trt_areas[0])    
print('area per size class: ',trt_props[0])
print('sizes: ', sizes[0])
print('units per size class: ', units[0])
print('radii per trt size: ',radii[0])
print('\n')

# for each scenario, make the treated landscape raster 
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")

# make an ee.ImageCollection with specified path
img_coll_p = f"projects/aff-treatments/assets/scenarios_{SCENARIOS}_{today_string}"
os.popen(f"earthengine create collection {img_coll_p}").read()

trt_imgs = []
for i in list(range(SCENARIOS)):
    trt_img = ee_treatments(units[i],radii[i]).set('scenario',i+1)
    export_img(trt_img,img_coll_p,aoi) #export image to image collection
    #break



# # making one works..
trt_img_example = ee_treatments(units[5],radii[5]).set('scenario',5+1)
# print(trt_img.getInfo())

Example Scenario
Total Area of AOI (ac):  245278
scenario pct area:  0.1747
total area (ac) to treat:  42850.0666
area per size class:  [14226.5389, 17624.6265, 10998.9012]
sizes:  [325, 125, 175]
units per size class:  [44, 141, 63]
radii per trt size:  [645, 400, 474]


Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario1
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario2
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario3
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario4
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario5
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario6
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario7
Export Started for projects/aff-treatments/assets/scenarios_60_20220719/scenario8
Export Started for projects/aff-treatments/assets/scenarios_60_20220719

In [6]:
Map = geemap.Map()

# Map.addLayer(parcels,{},'parcels')
# Map.addLayer(NIparcels,{},'NIP parcels')
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi,12)

pal = {'min':0,'max':1,'palette':['black','white']}
#print(trt_imgs.first().propertyNames().getInfo())
# Map.addLayer(trt_imgs.sort('scenario',False).first(), pal, 'final trt landscapes')
Map.addLayer(trt_img, pal, 'first scenario')
Map.addLayer(trt_img_example, pal, 'other random scneario')



Map

Map(center=[38.06601024718689, -120.26096170747162], controls=(WidgetControl(options=['position', 'transparent…